# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Maps, dictionaries and sets”

`Map` in F# is the _native_ F# equivalent of implementations of `IDictionary` that can be interchanged across .NET languages.

We can quickly obtain an immutable collection of key-value tuples from `List`:


In [ ]:
let inventory =
    [
        ("Apples", 0.33);
        ("Oranges", 0.23);
        ("Bananas", 0.45)
    ]
    |> Map.ofList

inventory |> printf "%A"

map [("Apples", 0.33); ("Bananas", 0.45); ("Oranges", 0.23)]

In F#, the `.` symbol is used to lookup members [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/symbol-and-operator-reference/#symbols-used-in-member-lookup-and-slice-expressions)]:

In [ ]:
inventory.["Oranges"]

0.23

Calling `Map.add` or `Map.remove` on this immutable collection allocates a new collection:

In [ ]:
let inventory2 =
    inventory
    |> Map.add "Pineapples" 0.87
    |> Map.remove "Apples"

inventory |> printf "%A\n"
inventory2 |> printf "%A"

map [("Apples", 0.33); ("Bananas", 0.45); ("Oranges", 0.23)]
map [("Bananas", 0.45); ("Oranges", 0.23); ("Pineapples", 0.87)]

>Importantly[,] calling `Add` on a `Map` that already contains the key won’t throw an exceptions. Instead, it’ll replace the old value with the new one as it creates the new `Map`…

In [ ]:
let inventory3 =
    inventory2
    |> Map.add "Pineapples" 1.07

inventory2 |> printf "%A\n"
inventory3 |> printf "%A"

map [("Bananas", 0.45); ("Oranges", 0.23); ("Pineapples", 0.87)]
map [("Bananas", 0.45); ("Oranges", 0.23); ("Pineapples", 1.07)]

However, accessing a key not in `Map` will throw `KeyNotFoundException`. Using `Map.tryFind` [📖 [docs](KeyNotFoundException )] can be useful to avoid such exceptions:


In [ ]:
let value =
    inventory2
    |> Map.tryFind "Apples"

value = None

True

## dictionaries

The word _dictionary_ in the world of F# means you intend to interoperate with `IDictionary`, making its instance mutable or immutable.

Making a mutable dictionary is intuitive from a C# point of view:



In [ ]:
open System.Collections.Generic

let inventory = Dictionary<string, float>()

inventory.Add("Apples", 0.33)
inventory.Add("Oranges", 0.23)
inventory.Add("Bananas", 0.45)

inventory.Remove("Apples")

inventory

key,value
Oranges,0.23
Bananas,0.45


And it must be said that F# sees nothing but a parameterless constructor for `Dictionary<_,_>` because, in C#, the following is possible:


In [ ]:
var inventory = new Dictionary<string, float>(new []
    {
        new KeyValuePair<string, float>("Apples", 0.33f),
        new KeyValuePair<string, float>("Oranges", 0.23f),
        new KeyValuePair<string, float>("Bananas", 0.45f),
    });

inventory.Remove("Apples");

inventory

key,value
Oranges,0.23
Bananas,0.45


Moreover, in C#, there is the collection initializer syntax [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/object-and-collection-initializers#collection-initializers)] which is more succinct:


In [ ]:
var inventory2 = new Dictionary<string, float>
    {
        { "Apples", 0.33f },
        { "Oranges", 0.23f },
        { "Bananas", 0.45f },
    };

inventory2.Remove("Apples");

inventory2

key,value
Oranges,0.23
Bananas,0.45


### immutable dictionaries with `dict`

I assume that the use of `IDictionary` in F# is so limited because its use is not encouraged. One way to transition out of its use is to use the `dict` function:

In [ ]:
let inventory =
    [
        ("Apples", 0.33);
        ("Oranges", 0.23);
        ("Bananas", 0.45)
    ]
    |> dict

inventory

key,value
Apples,0.33
Oranges,0.23
Bananas,0.45


Now this instance of `IDictionary` was generated in an expression as succinct as our last C# statements above. However, this instance is immutable. We can see this expressed violently with the thrown `NotSupportedException`:


In [ ]:
inventory.Remove("Apples")

Error: System.NotSupportedException: This value cannot be mutated
   at Microsoft.FSharp.Core.ExtraTopLevelOperators.DictImpl`3.System.Collections.Generic.IDictionary<'Key, 'T>.Remove(TKey _arg5) in D:\workspace\_work\1\s\src\fsharp\FSharp.Core\fslib-extra-pervasives.fs:line 81
   at <StartupCode$FSI_0029>.$FSI_0029.main@()

We can get ourselves back into mutable trouble with this foot gun:


In [ ]:
let inventory = inventory |> Dictionary

inventory.Remove("Apples")

inventory

key,value
Oranges,0.23
Bananas,0.45


@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.246201+da749355d416da20e634e5c80073b92356b57e0eBuild date: 2021-09-12T07:21:44.0000000Zhttps://github.com/dotnet/interactive
